## installation des dependances

In [29]:
!pip install datasets[audio]
!pip install evaluate
!pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-c_uz3fnn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-c_uz3fnn
  Resolved https://github.com/huggingface/transformers.git to commit 1ae4d917ed3badbdb1ffc167e0529f5a6d3c080d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## connection a hugging face

In [1]:
from huggingface_hub import notebook_login
notebook_login()

## Importation et traitement des données

In [2]:
from datasets import load_dataset
import subprocess

dataset = load_dataset(
    "DigitalUmuganda/ASR_Fellowship_Challenge_Dataset",
    data_files={
        "train": ["train_tarred/sharded_manifests_with_image/manifest_0.json"],
        "validation": f"val_tarred/sharded_manifests_with_image/manifest_*.json",
        "test": f"test_tarred/sharded_manifests_with_image/manifest_*.json"},
)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['recorder_id', 'project', 'image_filepath', 'audio_filepath', 'raw_text', 'transcriber_id', 'text', 'duration', 'LUFS', 'gender', 'age_group', 'location', 'shard_id', 'image_shard_id'],
        num_rows: 6532
    })
    validation: Dataset({
        features: ['recorder_id', 'project', 'image_filepath', 'audio_filepath', 'raw_text', 'transcriber_id', 'text', 'duration', 'LUFS', 'gender', 'age_group', 'location', 'shard_id', 'image_shard_id'],
        num_rows: 1617
    })
    test: Dataset({
        features: ['recorder_id', 'project', 'image_filepath', 'audio_filepath', 'raw_text', 'transcriber_id', 'text', 'duration', 'LUFS', 'gender', 'age_group', 'location', 'shard_id', 'image_shard_id'],
        num_rows: 1569
    })
})

In [5]:
from huggingface_hub import hf_hub_download
import os

files_list = [
    "train_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz",
    "test_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz",
    "val_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz",
]

base_output = "/content/sample_data/audio_dir"
os.makedirs(base_output, exist_ok=True)

def resolve_output_dir(file_path):
    if file_path.startswith("train"):
        return os.path.join(base_output, "train")
    elif file_path.startswith("test"):
        return os.path.join(base_output, "test")
    elif file_path.startswith("val"):
        return os.path.join(base_output, "val")
    else:
        return os.path.join(base_output, "other")

for f in files_list:
    # Pick output folder automatically
    output_dir = resolve_output_dir(f)
    os.makedirs(output_dir, exist_ok=True)

    print(f"\n📥 Downloading: {f}")
    file_path = hf_hub_download(
        repo_id="DigitalUmuganda/ASR_Fellowship_Challenge_Dataset",
        filename=f,
        repo_type="dataset"
    )

    print(f"✔ Downloaded to: {file_path}")
    print(f"📦 Extracting into: {output_dir}")

    # Extract .tar.xz
    subprocess.run(
        ["tar", "-xJf", file_path, "-C", output_dir],
        check=True
    )

print("\n✅ All files downloaded and extracted successfully!")


📥 Downloading: train_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz


train_tarred/sharded_manifests_with_imag(…):   0%|          | 0.00/2.07G [00:00<?, ?B/s]

✔ Downloaded to: /root/.cache/huggingface/hub/datasets--DigitalUmuganda--ASR_Fellowship_Challenge_Dataset/snapshots/62bdcc03d140f249f904fa52cf54548d58cd8ead/train_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz
📦 Extracting into: /content/sample_data/audio_dir/train

📥 Downloading: test_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz


test_tarred/sharded_manifests_with_image(…):   0%|          | 0.00/522M [00:00<?, ?B/s]

✔ Downloaded to: /root/.cache/huggingface/hub/datasets--DigitalUmuganda--ASR_Fellowship_Challenge_Dataset/snapshots/62bdcc03d140f249f904fa52cf54548d58cd8ead/test_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz
📦 Extracting into: /content/sample_data/audio_dir/test

📥 Downloading: val_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz


val_tarred/sharded_manifests_with_image/(…):   0%|          | 0.00/416M [00:00<?, ?B/s]

✔ Downloaded to: /root/.cache/huggingface/hub/datasets--DigitalUmuganda--ASR_Fellowship_Challenge_Dataset/snapshots/62bdcc03d140f249f904fa52cf54548d58cd8ead/val_tarred/sharded_manifests_with_image/audio_shards/audio_0.tar.xz
📦 Extracting into: /content/sample_data/audio_dir/val

✅ All files downloaded and extracted successfully!


In [3]:
dataset_train = dataset["train"].select(range(1000))
dataset_val = dataset["validation"].select(range(300))
dataset_test = dataset["test"].select(range(300))

In [4]:
dataset_train = dataset_train.remove_columns(["recorder_id","project","image_filepath", "transcriber_id","LUFS","gender","age_group","location","shard_id","image_shard_id"])
dataset_val = dataset_val.remove_columns(["recorder_id","project","image_filepath", "transcriber_id","LUFS","gender","age_group","location","shard_id","image_shard_id"])
dataset_test = dataset_test.remove_columns(["recorder_id","project","image_filepath", "transcriber_id","LUFS","gender","age_group","location","shard_id","image_shard_id"])

In [5]:
dataset_train

Dataset({
    features: ['audio_filepath', 'raw_text', 'text', 'duration'],
    num_rows: 1000
})

## Chargement et préparation des données pour l'ASR (Reconnaissance Automatique de la Parole)

In [6]:
import random

for _ in range(5):
    idx = random.randint(0, len(dataset_train) - 1)
    print(f"Sample {idx}: {dataset_train[idx]['text']}")

Sample 411: imiti gakondo yifashishwa n'abantu bagiye batandukanye mu buzima bwabo bwa buri munsi iyo ukoresheje uyu muti neza bituma ubasha kugira ubuzima bwiza kandi buzira umuze
Sample 1: umugabo n'umugore bakora kwa muganga bashobora gukora inshingano zitandukanye ariko bose bakenera ikoranabuhanga ryo kwifashisha mu kuvura abarwayi
Sample 569: uru ni uruhurirane rw'abantu benshi abagabo n'abagore bose baricaye harimo abagabo bambaye amakoti ari mu iba ry'umukara bambaye amakaruvate mu imbere bambayemo amashati ari mu ibara ry'umweru hari n'umugore ufite mikoro mu ntoki inyuma yabo hari icyapa cyandikishijeho amagambo mu ibara ry'umweru
Sample 430: itsinda ry'abantu batanu barimo abagore bane n'umugabo bakikijwe n'amakoma bicaye baganira ku bintu bitandukanye
Sample 461: aya ni amoko y'imiti igiye itandukanye ikomoka ku bimera cyanga se imiti gakondo agiye ateretse mu ijage ndetse no mu mavaze aho iyi miti yose yifashishwa bagatunganya imiti maze bakavura abantu


In [7]:
import re
chars_to_remove_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):

  if isinstance(batch["text"], str):
    batch["text"] = re.sub(chars_to_remove_regex, '', batch["text"]).lower()
  return batch

In [8]:
dataset_train = dataset_train.map(remove_special_characters)
dataset_test = dataset_test.map(remove_special_characters)
dataset_val = dataset_val.map(remove_special_characters)


In [9]:
def extract_all_chars(batch):
  all_text = " ".join(map(str, filter(None, batch["text"])))
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [10]:
vocab_train = dataset_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_train.column_names)
vocab_test = dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_test.column_names)
vocab_val = dataset_val.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_val.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

## Tokenization for ASR

In [11]:
vocab_train["vocab"]

Column([['k', 'f', 'a', 'j', 'w', 'b', 'd', 'o', ' ', 's', 'l', 'v', 'g', 'p', 'i', 'y', 't', 'c', 'u', 'm', 'z', 'n', 'r', 'e', 'x', 'h']])

In [12]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [13]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

28

In [14]:
vocab_dict={"'": 1,
 '[PAD]': 29,
 '[UNK]': 28,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27,
 '|': 0}

In [15]:
target_lang = "kin"
new_vocab_dict = {target_lang: vocab_dict}

In [16]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(new_vocab_dict, vocab_file)

In [17]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", target_lang=target_lang)


In [18]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [19]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [23]:
import os

def fix_file_path(example, base_dir):

    filename = example["audio_filepath"]
    full_path = os.path.join(base_dir, filename)
    example["audio_filepath"] = full_path
    return example

## Préparation des données pour l'entraînement

In [24]:
dataset_train[0]["audio_filepath"]

'audio_1743051032-WWNK27VxsCWmqrKg17pTtQyEcsg1.webm'

In [25]:
dataset_train = dataset_train.map(
    lambda x: fix_file_path(x, "/content/sample_data/audio_dir/train"),
    desc="Correction des chemins audio"
)


Correction des chemins audio:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
dataset_val = dataset_val.map(
    lambda x: fix_file_path(x, "/content/sample_data/audio_dir/val"),
    desc="Correction des chemins audio"
)

Correction des chemins audio:   0%|          | 0/300 [00:00<?, ? examples/s]

In [27]:
dataset_test = dataset_test.map(
    lambda x: fix_file_path(x, "/content/sample_data/audio_dir/test"),
    desc="Correction des chemins audio"
)

Correction des chemins audio:   0%|          | 0/300 [00:00<?, ? examples/s]

In [28]:
dataset_train[0]

{'audio_filepath': '/content/sample_data/audio_dir/train/audio_1743051032-WWNK27VxsCWmqrKg17pTtQyEcsg1.webm',
 'raw_text': 'Abagabo babiri aho bari gusindagiza umuntu uri hagati, aho yagize ikibazo atari kubasha kugenda neza kugira ngo amugeze ahabugenewe bityo amenye.',
 'text': 'abagabo babiri aho bari gusindagiza umuntu uri hagati aho yagize ikibazo atari kubasha kugenda neza kugira ngo amugeze ahabugenewe bityo amenye',
 'duration': 13.32}

In [29]:
import librosa
import numpy as np

TARGET_SR = 16000

def add_audio_feature(example):
    """
    Charge l'audio depuis example["file_path"],
    convertit en mono + 16kHz,
    ajoute une colonne example["audio"].
    """
    audio_path = example["audio_filepath"]

    # Load audio (mono=True garantit 1 canal)
    wav, sr = librosa.load(audio_path, sr=None, mono=True)

    # Resample si nécessaire
    if sr != TARGET_SR:
        wav = librosa.resample(wav, orig_sr=sr, target_sr=TARGET_SR)

    # Store final audio
    example["audio"] = wav.astype(np.float32)

    return example

In [30]:
dataset_val = dataset_val.map(
    add_audio_feature,
    desc="Loading audio 16k"
)

Loading audio 16k:   0%|          | 0/300 [00:00<?, ? examples/s]

/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/us

In [31]:
dataset_train = dataset_train.map(
    add_audio_feature,
    desc="Loading audio 16k"
)

Loading audio 16k:   0%|          | 0/1000 [00:00<?, ? examples/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(

In [32]:
dataset_test = dataset_test.map(
    add_audio_feature,
    desc="Loading audio 16k"
)

Loading audio 16k:   0%|          | 0/300 [00:00<?, ? examples/s]

/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3283411409.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=None, mono=True)
/us

In [33]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio, sampling_rate=16000)
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["text"]).input_ids
    return batch


In [ ]:
dataset_train = dataset_train.map(prepare_dataset, remove_columns=dataset_train.column_names)

In [35]:
dataset_val = dataset_val.map(prepare_dataset, remove_columns=dataset_test.column_names)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [36]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [37]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

## Métriques et entraînement

In [38]:
from evaluate import load

wer_metric = load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [39]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/mms-1b-all",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/1096 [00:00<?, ?it/s]

Wav2Vec2ForCTC LOAD REPORT from: facebook/mms-1b-all
Key            | Status   |                                                                                          
---------------+----------+------------------------------------------------------------------------------------------
lm_head.bias   | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([154]) vs model:torch.Size([32])            
lm_head.weight | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([154, 1280]) vs model:torch.Size([32, 1280])
lm_head.bias   | MISC     | 'Linear' object has no attribute 'param_name'
Error when processing parameter lm         
lm_head.weight | MISC     | 'Linear' object has no attribute 'param_name'
Error when processing parameter lm         

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight.
- MISC	:originate from the conversion scheme


In [43]:
model.init_adapter_layers()



In [44]:
model.freeze_base_model()

adapter_weights = model._get_adapters()
for param in adapter_weights.values():
    param.requires_grad = True


In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/sample_data/output",
  group_by_length=True,
  per_device_train_batch_size=32,
  eval_strategy="steps",
  num_train_epochs=4,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=200,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-3,
  warmup_steps=100,
  save_total_limit=2,
  push_to_hub=False,
)


In [53]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    processing_class=processor
)


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 31, 'bos_token_id': 30}.


In [ ]:
from safetensors.torch import save_file as safe_save_file
from transformers.models.wav2vec2.modeling_wav2vec2 import WAV2VEC2_ADAPTER_SAFE_FILE
import os

adapter_file = WAV2VEC2_ADAPTER_SAFE_FILE.format(target_lang)
adapter_file = os.path.join(training_args.output_dir, adapter_file)

safe_save_file(model._get_adapters(), adapter_file, metadata={"format": "pt"})


In [ ]:
input_dict = processor(dataset_test[0]["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


## Résultats et Analyse

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(dataset_test[0]["text"])